# 选题介绍 - 快速排序

## 小组成员

- 计1504 41416058 徐经纬
- 计1504 41524206 李凯
- 计1504 41524208 李鸿卓
- 计1504 41524201 朱柯佳
- 计1502 61562034 艾万


## 朴素快速排序算法


众所周知的基于分治思想的排序算法，朴素的实现上，最优时间复杂度为$O(nlogn)$，最坏时间复杂度可以被卡到$O(n^2)$

以下针对整形数组来介绍和探究快速排序算法的应用和扩展

主要算法步骤：

- 从数组中选择一个目标值aim，按照下面的方式划分数组：
    
    - 所有小于aim的元素都划分到aim的左侧
    
    - 所有不小于aim的元素都划分到aim的右侧

- 递归对左右两部分分别进行上一步操作，直至整个数组处理完成

### 一个简单的例子

    8 5 7 3 9    - 选择目标值为7

    7 5 8 3 9    - 将目标位置和当前最左边位置互换，令left=1(val=5), right=5(val=9)

    7 5 8 3 9    - 右移left，左移right至满足交换条件：left=3(val=8), right=4(val=3), 交换两个位置的值

    7 5 3 8 9    - left=4 > right=3, 结束交换，交换right位置和最左边位置

    3 5 7 8 9    - 对[3 5]和[8 9]两部分递归进行处理
    
    3 5 7 8 9    - 完成排序

### 优化

主要算法步骤：

- 从数组中选择一个目标值aim，按照下面的方式划分数组：
    
    - 所有小于aim的元素都划分到aim的左侧
    
    - 所有不小于aim的元素都划分到aim的右侧

- 递归对左右两部分分别进行上一步操作，直至整个数组处理完成

主要优化点：

- [x] 目标位置选取优化：对于固定选取最左边位置，完全逆序数据会被卡到最差复杂度 => 结论：固定目标位置的选取都是不优秀的
- [x] 划分方式优化：对于朴素二分方式，完全相同数据会被卡到最差复杂度 => 结论：对于存在大量相同数据的数据集，朴素二分方式是不优秀的


```cpp
using namespace std;
typedef pair<int, int> P;

P partition(int a[], int l, int r) {
	if (l >= r) return P(l, r);
	int cur = l;
	int left = l;
	int right = r;
	int p = rand() % (r - l + 1);
	int aim = a[l + p];
	swap(a[l + p], a[l]);
	while (cur <= right) {
		if (a[cur] == aim) cur++;
		else if (a[cur] < aim) swap(a[cur++], a[left++]);
		else swap(a[cur], a[right--]);
	}
	return P(left, right);
}

void qsort(int a[], int l, int r) {
	if (l >= r) return;
	P ret = partition(a, l, r);
    int left = ret.first, right = ret.second;
	qsort(a, l, left - 1);
	qsort(a, right + 1, r);
}
```

- $a[l...left - 1] < aim$
- $a[left...right] = aim$
- $a[right+ 1...r] > aim$

In [43]:
import subprocess
def run(method, workers, testSize, seed, compiled=True):
    if method not in ['naive', 'openmp', 'mpi']:
        raise ValueError(method)
    script = './run-{}.sh'.format(method)
    cmd = [script, '-w {}'.format(workers), '-n {}'.format(testSize), '-s {}'.format(seed), ]
    if not compiled:
        cmd.append('-c')
    return float(subprocess.check_output(cmd))

In [58]:
import plotly.plotly
from plotly.offline import iplot
import plotly.graph_objs as go

SEEDS = [i - 1 for i in range(10)]
TESTSIZES = [10**i for i in range(1, 9)]

def trace1():
    testSize = int(1e7)
    seed = SEEDS
    y = []
    for i in seed:
        timeInSec = run('naive', 1, testSize, i)
        y.append(timeInSec)
    return go.Scatter(x=seed, y=y, mode='lines+markers', name='fixed test size')

def trace2():
    testSize = TESTSIZES
    seed = 123456
    y = []
    for i in testSize:
        timeInSec = run('naive', 1, i, seed)
        y.append(timeInSec)
    return go.Scatter(x=testSize, y=y, mode='lines+markers', name='fixed seed')

trace_naive_testsize = trace1()
trace_naive_seed = trace2()
iplot({
    "data": [trace_naive_testsize, trace_naive_seed],
    "layout": go.Layout(title="plot of quick sort", xaxis=dict(title='x axis'))
})

TypeError: Object of type bytes is not JSON serializable

# OpenMp - 快速排序

```cpp
void qsort(int a[], int l, int r) {
	if (l >= r) return;
	P ret = partition(a, l, r);
	qsort(a, l, ret.first - 1);
	qsort(a, ret.second + 1, r);
}
```

# OpenMp - 快速排序

```cpp
void qsort(int a[], int l, int r) {
	if (l >= r) return;
	P ret = partition(a, l, r);
	int left = ret.first, right = ret.second;
	{
		#pragma omp task firstprivate(a, l, left)
		{
			qsort(a, l, left - 1);
		}
		#pragma omp task firstprivate(a, right, r)
		{
			qsort(a, right + 1, r);
		}
	}
}

/////////////////

omp_set_nested(1);

omp_set_num_threads(num_of_threads);

#pragma omp parallel shared(data, n)
{
    #pragma omp single nowait
    {
        qsort(data, 0, n - 1);
    }
}
```

### section | task

### firstprivate lastprivate

### single nowait



# 测试方案

以元素进入task前的值作为初值

测试方案：固定机器
method: naive openmp mpi
workers: 1 2 3 4 5 6 7 8
testSize: 1e1 1e2 1e3 1e4 1e5 1e6 1e7 1e8
seed: -1(reverse) 0(equ) 1 2 3 4 5 6

a) method:
	1. naive:
		p1: testSize=1e7 seed=__all__
		p2: seed=1 testSize=__all__

	2. openmp or mpi:
		p3: seed=-1 testsize=1e7 workers=__all__
		p4: seed=0  testsize=1e7 workers=__all__
		p5: seed=2  testsize=1e7 workers=__all__
		
		p6: seed=3




In [16]:
run('naive', 4, 10000000, 4516)

b'4.824022\n'

In [33]:
run("mpi", 2, 3000000, 4516)

b'1.341798\n'

In [41]:
run("openmp", 6, 3000000, 4516)

b'1.579056\n'